In [1]:
import pygame
import random
import math

pygame.init()
pygame.joystick.init()  # Initialize the joystick module

# Original window size for scaling reference
OLD_WINDOW_SIZE = (600, 600)

# New window size
WINDOW_SIZE = (1200, 800)

# Scaling factors
SCALING_FACTOR_X = WINDOW_SIZE[0] / OLD_WINDOW_SIZE[0]
SCALING_FACTOR_Y = WINDOW_SIZE[1] / OLD_WINDOW_SIZE[1]
SCALING_FACTOR = (SCALING_FACTOR_X + SCALING_FACTOR_Y) / 2  # Average scaling factor

# Constants (adjusted with scaling factor)
WHITE = (255, 255, 255)
BLACK = (0, 0, 0)
RED = (255, 0, 0)
GREEN = (0, 200, 0)
BLUE = (0, 0, 255)
YELLOW = (255, 255, 0)
FONT_COLOR = (0, 0, 0)
FONT_SIZE = int(24 * SCALING_FACTOR)
ARROW_LENGTH = int(60 * SCALING_FACTOR)
NUM_GOALS = 3
OBSTACLE_RADIUS = int(20 * SCALING_FACTOR)  
COLLISION_BUFFER = int(5 * SCALING_FACTOR)  
GRAY = (128, 128, 128)  
ENABLE_OBSTACLES = True
MAX_SPEED = 3 * SCALING_FACTOR

# Define dot/goal radii for detection
DOT_RADIUS = int(30 * SCALING_FACTOR)
TARGET_RADIUS = int(10 * SCALING_FACTOR)
GOAL_DETECTION_RADIUS = DOT_RADIUS + TARGET_RADIUS

targets = []
obstacles = []

# Set up display
screen = pygame.display.set_mode(WINDOW_SIZE)
pygame.display.set_caption("2D Environment with Path Prediction")

# Load font for rendering text
font = pygame.font.Font(None, FONT_SIZE)

START_POS = [WINDOW_SIZE[0] // 2, WINDOW_SIZE[1] // 2]
dot_pos = START_POS.copy()  # The dot's position
gamma = 0.5  # Arbitration function
reached_goal = False

# Create multiple targets
for _ in range(NUM_GOALS):
    targets.append([random.randint(0, WINDOW_SIZE[0]), random.randint(0, WINDOW_SIZE[1])])

current_target_idx = 0

# Initialize joystick
joystick = None
if pygame.joystick.get_count() > 0:
    joystick = pygame.joystick.Joystick(0)
    joystick.init()
    print("Joystick initialized:", joystick.get_name())
else:
    print("No joystick detected.")

def distance(pos1, pos2):
    return math.hypot(pos1[0] - pos2[0], pos1[1] - pos2[1])

def line_circle_intersection(start, end, circle_center, radius):
    dx = end[0] - start[0]
    dy = end[1] - start[1]
    cx = circle_center[0] - start[0]
    cy = circle_center[1] - start[1]
    l2 = dx*dx + dy*dy
    if l2 == 0:
        return distance(start, circle_center) <= radius
    t = max(0, min(1, (cx*dx + cy*dy) / l2))
    projection_x = start[0] + t * dx
    projection_y = start[1] + t * dy
    return distance([projection_x, projection_y], circle_center) <= radius

def check_collision(pos, new_pos):
    if not ENABLE_OBSTACLES:
        return False
    for obstacle_pos in obstacles:
        if line_circle_intersection(pos, new_pos, obstacle_pos, OBSTACLE_RADIUS + COLLISION_BUFFER):
            return True
    return False

def predict_human_target(human_input):
    global current_target_idx

    # If close enough to the current target, stick to it
    dist_to_current = distance(dot_pos, targets[current_target_idx])
    close_threshold = GOAL_DETECTION_RADIUS * 2
    if dist_to_current < close_threshold:
        return current_target_idx

    if human_input[0] == 0 and human_input[1] == 0:
        return current_target_idx

    best_score = float('-inf')
    best_target_idx = current_target_idx

    for i, target in enumerate(targets):
        to_target_dx = target[0] - dot_pos[0]
        to_target_dy = target[1] - dot_pos[1]

        movement_mag = math.hypot(human_input[0], human_input[1])
        target_mag = math.hypot(to_target_dx, to_target_dy)

        if movement_mag == 0 or target_mag == 0:
            continue

        # Reduced alignment weighting, increased distance factor
        alignment = (human_input[0] * to_target_dx + human_input[1] * to_target_dy) / (movement_mag * target_mag)
        dist = distance(dot_pos, target)
        max_dist = math.hypot(WINDOW_SIZE[0], WINDOW_SIZE[1])
        distance_factor = 1 - (dist / max_dist)

        # More focus on proximity to the goal than orientation
        score = (alignment * 0.3) + (distance_factor * 0.7)

        if score > best_score:
            best_score = score
            best_target_idx = i

    return best_target_idx

def generate_obstacles():
    obstacles.clear()
    if not ENABLE_OBSTACLES:
        return
    # Add code if you want to generate obstacles

def generate_targets():
    targets.clear()
    for _ in range(NUM_GOALS):
        while True:
            pos = [random.randint(0, WINDOW_SIZE[0]),
                   random.randint(0, WINDOW_SIZE[1])]
            valid_position = True
            if ENABLE_OBSTACLES:
                for obstacle_pos in obstacles:
                    if distance(pos, obstacle_pos) < OBSTACLE_RADIUS * 1.5:
                        valid_position = False
                        break
            if valid_position:
                targets.append(pos)
                break

def move_dot(human_input):
    global dot_pos, gamma, reached_goal, current_target_idx

    h_dx, h_dy = human_input
    h_mag = math.hypot(h_dx, h_dy)
    h_dir = [h_dx / h_mag, h_dy / h_mag] if h_mag > 0 else [0, 0]

    target_pos = targets[current_target_idx]
    w_dx = target_pos[0] - dot_pos[0]
    w_dy = target_pos[1] - dot_pos[1]
    w_mag = math.hypot(w_dx, w_dy)
    w_dir = [w_dx / w_mag, w_dy / w_mag] if w_mag > 0 else [0, 0]

    x_dir_x = gamma * w_dir[0] + (1 - gamma) * h_dir[0]
    x_dir_y = gamma * w_dir[1] + (1 - gamma) * h_dir[1]
    x_dir_mag = math.hypot(x_dir_x, x_dir_y)
    x_dir = [x_dir_x / x_dir_mag, x_dir_y / x_dir_mag] if x_dir_mag > 0 else [0, 0]

    input_mag = min(max(h_mag / MAX_SPEED, 0), 1)
    step_size = MAX_SPEED * input_mag

    new_x = dot_pos[0] + x_dir[0] * step_size
    new_y = dot_pos[1] + x_dir[1] * step_size

    if not check_collision(dot_pos, [new_x, new_y]):
        dot_pos[0] = max(0, min(WINDOW_SIZE[0], new_x))
        dot_pos[1] = max(0, min(WINDOW_SIZE[1], new_y))

    # Register goal if within radius (treat every goal as final)
    dist_to_goal = distance(dot_pos, target_pos)
    if dist_to_goal < GOAL_DETECTION_RADIUS:
        reached_goal = True
        pygame.time.set_timer(pygame.USEREVENT, 1000)

    return h_dir, w_dir, x_dir

def reset():
    global dot_pos, reached_goal, current_target_idx, gamma

    dot_pos = START_POS.copy()
    reached_goal = False
    current_target_idx = 0
    gamma = 0.5

    generate_obstacles()
    generate_targets()
    pygame.time.set_timer(pygame.USEREVENT, 0)

def draw_arrow(surface, color, start_pos, direction, length=ARROW_LENGTH):
    dx, dy = direction
    if dx == 0 and dy == 0:
        return
    dir_length = math.hypot(dx, dy)
    dx /= dir_length
    dy /= dir_length

    end_x = start_pos[0] + dx * length
    end_y = start_pos[1] + dy * length

    pygame.draw.line(surface, color, start_pos, (end_x, end_y), int(2 * SCALING_FACTOR))

    arrow_size = 7 * SCALING_FACTOR
    angle = math.atan2(dy, dx)
    arrow1_x = end_x - arrow_size * math.cos(angle + math.pi/6)
    arrow1_y = end_y - arrow_size * math.sin(angle + math.pi/6)
    arrow2_x = end_x - arrow_size * math.cos(angle - math.pi/6)
    arrow2_y = end_y - arrow_size * math.sin(angle - math.pi/6)

    pygame.draw.line(surface, color, (end_x, end_y), (arrow1_x, arrow1_y), int(2 * SCALING_FACTOR))
    pygame.draw.line(surface, color, (end_x, end_y), (arrow2_x, arrow2_y), int(2 * SCALING_FACTOR))

def render(h_dir, w_dir, x_dir):
    screen.fill(WHITE)

    if ENABLE_OBSTACLES:
        for obstacle_pos in obstacles:
            pygame.draw.circle(screen, GRAY, (int(obstacle_pos[0]), int(obstacle_pos[1])), OBSTACLE_RADIUS)

    for i, target in enumerate(targets):
        pygame.draw.circle(screen, YELLOW, (int(target[0]), int(target[1])), TARGET_RADIUS)
        num_text = font.render(str(i+1), True, BLACK)
        screen.blit(num_text, (target[0] - 5, target[1] - 12))

    current_target = targets[current_target_idx]
    pygame.draw.circle(screen, BLACK, (int(current_target[0]), int(current_target[1])), TARGET_RADIUS+2, int(2 * SCALING_FACTOR))
    pygame.draw.circle(screen, BLACK, (int(dot_pos[0]), int(dot_pos[1])), DOT_RADIUS, int(2 * SCALING_FACTOR))

    if h_dir != [0,0]:
        draw_arrow(screen, BLUE, (int(dot_pos[0]), int(dot_pos[1])), h_dir, length=ARROW_LENGTH)
    if w_dir != [0,0]:
        draw_arrow(screen, GREEN, (int(dot_pos[0]), int(dot_pos[1])), w_dir, length=ARROW_LENGTH)
    if x_dir != [0,0]:
        draw_arrow(screen, RED, (int(dot_pos[0]), int(dot_pos[1])), x_dir, length=ARROW_LENGTH)

    gamma_text = font.render(f"Gamma: {gamma:.2f}", True, FONT_COLOR)
    screen.blit(gamma_text, (10, 10))
    formula_text = font.render(f"Movement = {gamma:.2f}W + {1-gamma:.2f}H", True, FONT_COLOR)
    screen.blit(formula_text, (10, 40))
    target_text = font.render(f"Current Target: {current_target_idx + 1}", True, FONT_COLOR)
    screen.blit(target_text, (10, 70))
    instructions_text = font.render("L2/R2 to adjust gamma, R to reset", True, FONT_COLOR)
    screen.blit(instructions_text, (10, 100))

    legend_y = WINDOW_SIZE[1] - int(100 * SCALING_FACTOR)
    legend_spacing = int(30 * SCALING_FACTOR)
    legend_items = [
        ("Green Arrow: Perfect Path (W)", GREEN),
        ("Blue Arrow: Human Movement (H)", BLUE),
        ("Red Arrow: Dot's Movement", RED)
    ]
    for i, (text, color) in enumerate(legend_items):
        label = font.render(text, True, color)
        screen.blit(label, (10, legend_y + i * legend_spacing))

    if reached_goal:
        reset_text = font.render("Goal Reached! Auto-resetting...", True, FONT_COLOR)
        screen.blit(reset_text, (150, 160))

    pygame.display.update()

running = True
clock = pygame.time.Clock()
gamma = 0.5

generate_obstacles()
generate_targets()

# Adjust these based on your device
AXIS_L2 = 4
AXIS_R2 = 5

while running:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False

        if event.type == pygame.KEYDOWN:
            if event.key == pygame.K_r:
                reset()

        if joystick and event.type == pygame.JOYBUTTONDOWN:
            if event.button == 2:  # Square button on PS5
                reset()

        # If USEREVENT is triggered by timer (after goal reached)
        if event.type == pygame.USEREVENT:
            reset()

    if not reached_goal:
        dx, dy = 0.0, 0.0
        keys = pygame.key.get_pressed()
        if keys[pygame.K_LEFT]:
            dx -= 1.0
        if keys[pygame.K_RIGHT]:
            dx += 1.0
        if keys[pygame.K_UP]:
            dy -= 1.0
        if keys[pygame.K_DOWN]:
            dy += 1.0

        if joystick:
            axis_0 = joystick.get_axis(0)  
            axis_1 = joystick.get_axis(1)  
            deadzone = 0.1
            if abs(axis_0) > deadzone or abs(axis_1) > deadzone:
                dx = axis_0
                dy = axis_1
            else:
                dx = 0.0
                dy = 0.0

            l2_value = joystick.get_axis(AXIS_L2)
            r2_value = joystick.get_axis(AXIS_R2)

            l2_pressed = l2_value > 0.1
            r2_pressed = r2_value > 0

            if l2_pressed:
                gamma = max(0.0, gamma - 0.01)
            if r2_pressed:
                gamma = min(1.0, gamma + 0.01)

        if abs(dx) < 0.1 and abs(dy) < 0.1:
            dx, dy = 0.0, 0.0

        dx *= MAX_SPEED
        dy *= MAX_SPEED

        human_input = [dx, dy]
        current_target_idx = predict_human_target(human_input)
        h_dir, w_dir, x_dir = move_dot(human_input)

    else:
        h_dir, w_dir, x_dir = [0, 0], [0, 0], [0, 0]

    render(h_dir, w_dir, x_dir)
    clock.tick(60)

pygame.quit()


pygame 2.6.1 (SDL 2.28.4, Python 3.12.3)
Hello from the pygame community. https://www.pygame.org/contribute.html
Joystick initialized: DualSense Wireless Controller
